In [1]:
import numpy as np
x=np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

높이 28, 너비 28, 채널 1개, 데이터 10개

In [2]:
x[0].shape

(1, 28, 28)

In [3]:
x[1].shape

(1, 28, 28)

In [5]:
x[0,0] # 첫번째 데이터의 첫 채널의 공간 데이터

array([[0.41479975, 0.60000481, 0.52027069, 0.92012887, 0.8997539 ,
        0.26756375, 0.16990612, 0.92363083, 0.05511312, 0.71030531,
        0.74385012, 0.78342287, 0.78997223, 0.48127093, 0.25165233,
        0.30589073, 0.89213015, 0.0738587 , 0.54282665, 0.95224035,
        0.97693007, 0.06212494, 0.36007385, 0.66041465, 0.17659511,
        0.38320129, 0.41100826, 0.24723813],
       [0.48165417, 0.99817073, 0.50081448, 0.99765176, 0.94766931,
        0.77742088, 0.61779545, 0.15319711, 0.27651072, 0.66212488,
        0.05311268, 0.66471052, 0.85051597, 0.47685555, 0.94141565,
        0.33106873, 0.22613775, 0.69392879, 0.59129621, 0.56769971,
        0.4125796 , 0.11987934, 0.99355194, 0.16613546, 0.89681295,
        0.2262341 , 0.16349421, 0.96858533],
       [0.1821341 , 0.2819352 , 0.50667035, 0.71977666, 0.94890145,
        0.83367179, 0.54999903, 0.94607099, 0.54072369, 0.87250521,
        0.18595847, 0.21083319, 0.19370727, 0.18351898, 0.54087888,
        0.61809628, 0.7140

In [6]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col


합성곱 계층을 Convolution이라는 클래스로 구현해보자

In [1]:
class Convolution:
    def __inti__(self, w, b, stride=1, pad=0):
        self.w=w
        self.b=b
        self.stride=stride
        self.pad
    
    def forward(self, x):
        fn, c, fh, fw=self.w.shape
        n,c,h,w=x.shape
        out_h=int(1+(h+2*self.pad-fh)/self.stride)
        out_w=int(1+(w+2*self.pad-fw)/self.stride)
        
        col=im2col(x, fh, fw, self.stride, self.pad)
        col_w=self.w.reshape(fn, -1).T
        out=np.dot(col,col_w)+self.b
        
        out=out.reshpae(n, out_h, out_w, -1).transpose(0,3,1,2)
        
        return out

필터는 (FN, C, FH, FW)의 4차원 형상이다. FN은 필터 개수, C는 채널, FH는 필터 높이, FW는 필터 너비이다. 

        col=im2col(x, fh, fw, self.stride, self.pad)

        col_w=self.w.reshape(fn, -1).T
        
        out=np.dot(col,col_w)+self.b

이 부분에서 입력 데이터를 im2col로 전개하고 필터도 reshape를 사용해 2차원 배열로 전개한다. 이렇게 전개한 두 행렬의 내적을 구한다. 각 필터 블록을 1줄로 펼쳐 세우고 reshpae에 -1을 지정해서 다차원 배열의 원소 수가 변환 후에도 똑같이 유지되도록 적절히 묶어준다. 
마지막 출력 데이터를 적절한 형상으로 바꿔주는 것이 forward 구현의 마지막이다. 

### 7.4.4 풀링 계층 구현하기

In [2]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h=pool_h
        self.pool_w=pool_w
        self.stride=stride
        self.pad=pad
        
    def forward(self, x):
        n, c, h, w=x.shape
        out_h=int(1+(h-self.pool_h)/self.stride)
        out_w=int(1+(w-self.pool_w)/self.stride)
        
        col=im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)  #전개
        col=col.reshape(-1, self.pool_h*self.pool_w)
        
        out=np.max(col, axis=1) #최댓값
        
        out=out.reshape(n, out_h, out_w, c).transpose(0, 3, 1, 2)
        
        return out

## 7.5 cnn 구현하기

In [7]:
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient


class SimpleConvNet:
    """단순한 합성곱 신경망
    
    conv - relu - pool - affine - relu - affine - softmax
    
    Parameters
    ----------
    input_size : 입력 크기（MNIST의 경우엔 784）
    hidden_size_list : 각 은닉층의 뉴런 수를 담은 리스트（e.g. [100, 100, 100]）
    output_size : 출력 크기（MNIST의 경우엔 10）
    activation : 활성화 함수 - 'relu' 혹은 'sigmoid'
    weight_init_std : 가중치의 표준편차 지정（e.g. 0.01）
        'relu'나 'he'로 지정하면 'He 초깃값'으로 설정
        'sigmoid'나 'xavier'로 지정하면 'Xavier 초깃값'으로 설정
    """
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

초기화를 마친 후에는 추론을 수행하는 predict메서드와 손실함수의 값을 구하는 loss 메서드를 구현할 수 있다.

In [10]:
def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

def loss(self, x, t):
        """손실 함수를 구한다.
        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블
        """
        y = self.predict(x)
        return self.last_layer.forward(y, t)

x는 입력데이터, t는 정답 레이블이다. 추론을 수행할 때는 초기화 때 layers에 추가한 계층을 맨 앞에서부터 차례로 forward 메서드를 호출해서 다음 계층에 결과를 전달한다.
손실함수는 predict 메서드의 결과를 인수로 마지막 층의 forwoard 메서드를 호출한다.

#### 오차역전파법으로 기울기를 구하는 구현

In [11]:
 def gradient(self, x, t):
        """기울기를 구한다(오차역전파법).
        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블
        Returns
        -------
        각 층의 기울기를 담은 사전(dictionary) 변수
            grads['W1']、grads['W2']、... 각 층의 가중치
            grads['b1']、grads['b2']、... 각 층의 편향
        """
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads